# GPT Dataset

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import tiktoken


In [3]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(input_chunk))
    
    def __len__(self):
        return len(self.input_ids)  # Number of chunks
    

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128,
                         shuffle=True, drop_last=True, num_workers=0):
    
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create the dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create the dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                            drop_last=drop_last, num_workers=num_workers)
    
    return dataloader

In [6]:
with open("../ch05/01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

vocab_size = 50257
output_dim = 256
context_length = 1024


token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length)

In [7]:
next(iter(dataloader))  

[tensor([[ 5986,    30,  2011, 20136],
         [  903,   597,   517,    11],
         [14482,  1625,   503,   438],
         [  287,   257, 14005,  7872],
         [  262,  2063,    12,  2971],
         [ 1464,  9885,   345,   286],
         [  314, 16896,    13,   198],
         [ 1560,   607,  4808,  5562]]),
 tensor([[ 5986,    30,  2011, 20136],
         [  903,   597,   517,    11],
         [14482,  1625,   503,   438],
         [  287,   257, 14005,  7872],
         [  262,  2063,    12,  2971],
         [ 1464,  9885,   345,   286],
         [  314, 16896,    13,   198],
         [ 1560,   607,  4808,  5562]])]

In [19]:
tokenizer.decode(next(iter(dataloader))[0][0].tolist())

' house of cards.'